In [3]:
import pandas as pd
import fastdup
import shutil
import os 


def search_parameter(df: pd.DataFrame = None, parameter: dict = None) -> pd.DataFrame:
    
    if not len(parameter):
        return df
    
    df_columns = df.columns
    
    # validamos que el parametro este en el df, en caso de que no lo quitamos
    for key in parameter.copy():
        if key not in df_columns:
            
            print(f"{key} no se encuentra en el df.")
            del parameter[key]
    
    if not len(parameter):
        print("Ninguno de los parámetros proporcionados se encuentran en el df por lo tanto lo retornamos como estaba.")
        return df
    
    # validamos los valores de busqueda
    for key,value in parameter.copy().items():
        
        if isinstance(value, (str, int, float, bool)):
            
            search = df[ df[key] == value]
            if not len(search):
                print(f"No se encontro en valor '{value}' en la columna '{key}' por lo tanto no se va a buscar.")
                del parameter[key]
        
        elif isinstance(value, list):
            
            search = df[df[key].isin(value)]
            if not len(search):
                print(f"No se encontro los valores '{value}' en la columna '{key}' por lo tanto no se va a buscar.")
                del parameter[key]
        
        elif isinstance(value, dict):
            print(f"No se aceptan parametros de tipo {type(value).__name__}")
            del parameter[key]
    
    if not len(parameter):
        print("Ninguno de los parámetros proporcionados se encuentran en el df por lo tanto lo retornamos como estaba.")
        return df
    
    # si todo sale bien empezamos a realizar las busquedas en el df
    df_copy = df
    
    for key, value in parameter.items():
        
        if isinstance(value, (str, int, float, bool)):
            
            df_copy = df_copy[ df_copy[key] == value]
        
        elif isinstance(value, list):
            
            df_copy = df_copy[df_copy[key].isin(value)]
    
    # retornamos el df filtrado
    return df_copy


# df = pd.read_json("../data/json/Myntra__Marianfer_Cruz_20231219_195028.success.json")
# print(len(df))
# parameter = {
#     "color": ["White", "Brown"],
#     "brand": "BUCIK",
# }

# df = search_parameter(df, parameter)
# print(len(df))

In [2]:
def delete_directory_content(directory: str = None) -> str:
    
    try:
        
        for archivo in os.listdir(directory):
            
            ruta_completa = os.path.join(directory, archivo)
            
            if os.path.isfile(ruta_completa):
                os.remove(ruta_completa)
            elif os.path.isdir(ruta_completa):
                shutil.rmtree(ruta_completa)
                
        print(f"El contenido del directory '{directory}' ha sido eliminado correctamente.")
        
        return directory
        
    except FileNotFoundError:
        print(f"El directory '{directory}' no existe.")
        
    except Exception as e:
        print(f"Se produjo un error al eliminar el contenido del directory '{directory}': {str(e)}")

In [5]:
delete_directory_content("../trash/fastdup")

El contenido del directory '../trash/fastdup' ha sido eliminado correctamente.


'../trash/fastdup'